In [ ]:
# ===============================================
# 🧠 Curadoria Leve de Dados Numéricos – Etapa 1
# Seleciona projeto e lista arquivos válidos no bucket storage-unique
# ===============================================

import os
from minio import Minio
import ipywidgets as widgets
from IPython.display import display, clear_output
from pathlib import Path

# Conectar ao MinIO
minio_client = Minio(
    "minio:9000",
    access_key="admin",
    secret_key="senhasegura",
    secure=False
)

bucket_origem = "storage-unique"
extensoes_validas = [".csv", ".xlsx", ".xls", ".parquet"]

# --- Widget de seleção de projeto (prefix)
# Aqui simulamos prefixos válidos via os diretórios do MinIO
objetos = list(minio_client.list_objects(bucket_origem, recursive=True))
prefixos_disponiveis = sorted(set(obj.object_name.split("/")[0] for obj in objetos if "/" in obj.object_name))

prefix_dropdown = widgets.Dropdown(options=prefixos_disponiveis, description="Projeto:")
run_button = widgets.Button(description="🔍 Buscar Arquivos", button_style="primary")
output = widgets.Output()

def listar_arquivos(btn):
    with output:
        clear_output()
        prefix = prefix_dropdown.value
        print(f"🔎 Listando arquivos com prefixo '{prefix}' no bucket '{bucket_origem}'...")
        arquivos_validos = [
            obj.object_name for obj in objetos
            if obj.object_name.startswith(prefix + "/") and any(obj.object_name.endswith(ext) for ext in extensoes_validas)
        ]
        
        if not arquivos_validos:
            print("⚠️ Nenhum arquivo numérico válido encontrado para esse projeto.")
        else:
            print(f"✅ {len(arquivos_validos)} arquivos encontrados:")
            for arq in arquivos_validos:
                print(f"• {arq}")

run_button.on_click(listar_arquivos)

display(widgets.VBox([prefix_dropdown, run_button, output]))
